# Athens Drive Basketball History LLM

# Import dependencies

In [35]:
import pandas as pd
import sqlite3

In [36]:
data = pd.read_csv('data/Playoffs.csv')
data = data.head(15)
# Drop unneeded columns
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'year', 'Season_type', 'PLAYER_ID',
       'RANK', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'AST_TOV',
       'STL_TOV'],
      dtype='object')

In [37]:
data = data[['GP','PLAYER','FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'STL', 'TOV' ]]
data.head()


,GP,PLAYER,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,TOV
0,23,LeBron James,212,432,0.491,36,96,0.375,136,175,0.777,37,156,41,70
1,21,Tony Parker,167,365,0.458,11,31,0.355,87,112,0.777,14,54,24,53
2,21,Tim Duncan,151,321,0.470,0,1,0.000,79,98,0.806,54,160,18,42
3,19,Paul George,119,277,0.430,34,104,0.327,93,128,0.727,16,125,25,75
4,22,Dwyane Wade,144,315,0.457,1,4,0.250,60,80,0.750,38,64,38,58


In [38]:
numeric_columns = data[['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA']]
for col in numeric_columns:
    if col != "GP":
        data[f"single_game_{col}"] = round((data[col]/ data["GP"]),1)

data

,GP,PLAYER,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,OREB,DREB,STL,TOV,single_game_FGM,single_game_FGA,single_game_FG3M,single_game_FG3A,single_game_FTM,single_game_FTA
0,23,LeBron James,212,432,0.491,36,96,0.375,136,175,...,37,156,41,70,9.2,18.8,1.6,4.2,5.9,7.6
1,21,Tony Parker,167,365,0.458,11,31,0.355,87,112,...,14,54,24,53,8.0,17.4,0.5,1.5,4.1,5.3
2,21,Tim Duncan,151,321,0.470,0,1,0.000,79,98,...,54,160,18,42,7.2,15.3,0.0,0.0,3.8,4.7
3,19,Paul George,119,277,0.430,34,104,0.327,93,128,...,16,125,25,75,6.3,14.6,1.8,5.5,4.9,6.7
4,22,Dwyane Wade,144,315,0.457,1,4,0.250,60,80,...,38,64,38,58,6.5,14.3,0.0,0.2,2.7,3.6
5,12,Carmelo Anthony,126,310,0.406,17,57,0.298,77,87,...,11,68,13,31,10.5,25.8,1.4,4.8,6.4,7.2
6,11,Kevin Durant,112,246,0.455,22,70,0.314,93,112,...,7,92,14,43,10.2,22.4,2.0,6.4,8.5,10.2
7,19,Roy Hibbert,120,235,0.511,0,0,0.000,83,103,...,90,98,3,43,6.3,12.4,0.0,0.0,4.4,5.4
8,19,David West,115,249,0.462,0,3,0.000,72,94,...,41,103,13,45,6.1,13.1,0.0,0.2,3.8,4.9
9,21,Kawhi Leonard,115,211,0.545,23,59,0.390,31,49,...,50,140,37,23,5.5,10.0,1.1,2.8,1.5,2.3


In [39]:
data = data.drop(columns=['GP', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM','FTA', 'FT_PCT', 'OREB', 'DREB', 'STL', 'TOV'])
data.columns

Index(['PLAYER', 'single_game_FGM', 'single_game_FGA', 'single_game_FG3M',
       'single_game_FG3A', 'single_game_FTM', 'single_game_FTA'],
      dtype='object')

In [42]:
data = data.rename(columns={
    "PLAYER": "PLAYER NAME",
    "single_game_FGM": "FIELD GOALS MADE",
    "single_game_FGA": "FIELD GOALS ATTEMPTED",
    "single_game_FG3M": "THREE-POINT GOALS MADE",
    "single_game_FG3A": "THREE-POINT GOALS ATTEMPTED",
    "single_game_FTM": "FREE THROWS MADE",
    "single_game_FTA": "FREE THROWS ATTEMPTED"
})


In [43]:
# Create a SQLite database connection
conn = sqlite3.connect("team_stats.db")

# Save the DataFrame to a table named 'player_stats'
data.to_sql("player_stats", conn, if_exists="replace", index=False)

15

In [44]:
conn.close()